In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from io import StringIO
from datetime import timedelta

#prevents warnings from being printed
import warnings
warnings.filterwarnings('ignore')

In [3]:
sd = pd.read_csv('sd_requests.csv')
sd = sd.convert_dtypes()
sd.head()

,id,status,desc,date,depts,docs,poc,msgs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D..."
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ..."
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re..."
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."


In [4]:
sd.shape

(28848, 8)

### Add state and city column 

In [5]:
sd = sd.rename(columns = {'date': 'request date'})
sd = sd.fillna('')

sd['city'] = 'San Diego'
sd['state'] = 'CA'

### Convert msgs and docs column to dataframe for info extraction

In [6]:
csv_to_df = lambda csv: pd.read_csv(StringIO(csv)) if csv else None
sd['docs_csv'] = sd['docs'].apply(csv_to_df)
sd['msgs_csv'] = sd['msgs'].apply(csv_to_df)

sd['docs_csv'] = sd['docs_csv'].apply(lambda df: df.convert_dtypes().fillna('') if df is not None else None)
sd['msgs_csv'] = sd['msgs_csv'].apply(lambda df: df.convert_dtypes().fillna('') if df is not None else None)

In [7]:
sd['num docs'] = sd['docs_csv'].map(lambda df: len(df) if df is not None else 0)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


### Convert request dates into datetime objects

In [8]:
sd['request date'] = sd['request date'].str.extract(r'([A-z][a-z]+\s+\d{1,2},\s+\d{4})')
sd['request date'] = pd.to_datetime(sd['request date'], infer_datetime_format=True)

In [9]:
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


In [10]:
#number of open and closed requests for san diego
pd.value_counts(sd['status'])

CLOSED    28843
OPEN          5
Name: status, dtype: Int64

In [11]:
#pd.Series(sd['depts'].unique()).to_frame().to_csv('unique sd depts.csv')

### Check types of departments within dataframe

In [12]:
dept_count = sd.groupby('depts').count().reset_index()
dept_count = dept_count[['depts', 'id']].rename(columns = {'id': 'requests received'})
dept_count

,depts,requests received
0,,143
1,ADA Compliance & Accessibility,6
2,Airports,41
3,Animal Services,849
4,Assistant Chief Operating Officer,4
...,...,...
469,"Transportation, Performance & Analytics",6
470,"Transportation, Public Records Administration",21
471,"Transportation, Public Records Administration ...",1
472,"Transportation, Stormwater",1


### Not every message has a government code. We will display only the ones where govt code is not null

In [15]:
def get_code(msgs):
    a = []
    if re.findall("Government Code section (.*)", msgs) != a:
        return str(re.findall("Government Code section (.[0-9]*\(.\))", msgs)).strip("[").strip("]").strip('\'').strip("§")

In [17]:
sd["Gov Code"] = sd["msgs"].apply(get_code)

In [19]:
sd[sd["Gov Code"].notnull()]

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code
8,15-1818,CLOSED,Incident and supplemental reports for: August ...,2015-12-07,Public Records Administration,,Lea Fields-Bernard,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,6254(c)
30,15-1840,CLOSED,Writings and communications related to Sai Kir...,2015-12-09,Deputy Chief Operating Officer - Infrastructur...,"title,link Letter to Finch (CCPRA 2015-1840).p...",Jacqueline Palmer,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,title \ 0 Request ...,2,6254(k)
39,15-1849,CLOSED,Communications and or calendar appointments be...,2015-12-10,Public Records Administration,,Lea Fields-Bernard,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,None,title \ 0 Request ...,0,6254(a)
49,15-1859,CLOSED,All records related to the demographic study c...,2015-12-11,Police,,Humberto Hernandez,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,None,title \ 0 Request ...,0,6254(c)
55,15-1865,CLOSED,Records related to the stadium EIR including s...,2015-12-14,Public Records Administration,"title,link Copy of S16025 Labor Details 12-16-...",Lea Fields-Bernard,"title,item,time ""Request Closed Public"",Govern...",San Diego,CA,...,title \ 0 R...,2,6254(c)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28835,21-5530,CLOSED,"Dear Custodian of Records, Under the Californ...",2021-10-26,Police,,Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,None,title \ 0 Requ...,0,6254(f)
28839,21-5550,CLOSED,I request any police reports involving inciden...,2021-10-27,Police,,Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,None,title \ 0 Requ...,0,6254(k)
28843,21-5579,CLOSED,"Hello, I would like a copy of the report from ...",2021-10-28,Animal Services,,Lori Hernandez,"title,item,time ""Request Published Public"",,""N...",San Diego,CA,None,title \ 0 Requ...,0,6254(f)
28845,21-5584,CLOSED,request for call for service 2110020816,2021-10-28,Police,"title,link 2110020816_Redacted.pdf,https://san...",Lori Hernandez,"title,item,time ""Request Published Public"",,""N...",San Diego,CA,title ...,title \ 0 Requ...,1,6254(f)


### Pull out response from each message in a new column

In [20]:
def get_response(msgs):
    return re.findall("Public\",.*\".*\n.{5,}\.", msgs)

In [22]:
sd["Response"] = sd["msgs"].apply(get_response)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[]
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[]
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[]
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[]
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[]


### Get the Submission Time 

In [23]:
def get_submission_time(msgs):
    return str(re.findall("Request received via web,(.*[0-9]*20..,.*:{1}..[pa]{1}m{1})", msgs)).strip("[").strip(
        "]").strip("'").strip("\"")

In [25]:
sd["Submission Time (str)"] = sd["msgs"].apply(get_submission_time)

sd["Submission Time (dt)"] = pd.to_datetime(sd["Submission Time (str)"])

sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt)
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[],"December 7, 2015, 5:08pm",2015-12-07 17:08:00
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:22pm",2015-12-07 17:22:00
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[],"December 7, 2015, 5:29pm",2015-12-07 17:29:00
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:33pm",2015-12-07 17:33:00
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[],"December 7, 2015, 5:39pm",2015-12-07 17:39:00


### Filter for only police requests

In [39]:
sd_police = sd[sd['depts'].str.contains("(?i)police|sheriff|public safety", case = False)]

### Getting the  Closing Times

In [79]:
def extract_time(tbl):
    d = {8: True, -1: False}
    closed_df = tbl[tbl['title'].str.find('Closed').map(d)]
    closing = closed_df['time'].str.extract(r'([A-z][a-z]+\s+\d{1,2},\s+\d{4}, ([0-1]?[0-9]|2[0-3]):[0-5][0-9][a|p]m)')
    return closing.iloc[0, 0]

In [80]:
sd_police['Closing Time (str)'] = sd_police['msgs_csv'].apply(extract_time)
sd_police.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time,close,closing
7,15-1817,CLOSED,File materials related to: P12010041171 P12050...,2015-12-07,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",02c. R...",San Diego,CA,None,title ...,0,None,[],"December 8, 2015, 11:40am",2015-12-08 11:40:00,"December 21, 2015, 10:48am","December 21, 2015, 10:48am","December 21, 2015, 10:48am"
49,15-1859,CLOSED,All records related to the demographic study c...,2015-12-11,Police,,Humberto Hernandez,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,None,title \ 0 Request ...,0,6254(c),[],"December 14, 2015, 6:07pm",2015-12-14 18:07:00,"January 12, 2016, 10:06am","January 12, 2016, 10:06am","January 12, 2016, 10:06am"
65,15-1875,CLOSED,Reports related to incident of 11/11/2015 wher...,2015-12-15,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",""02. R...",San Diego,CA,None,title \ 0 R...,0,None,"[Public"",""02. Released\nAll responsive documen...","December 16, 2015, 11:32am",2015-12-16 11:32:00,"January 13, 2016, 7:40am","January 13, 2016, 7:40am","January 13, 2016, 7:40am"
81,15-1891,CLOSED,Communications by the City of San Diego regard...,2015-12-17,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,None,title ...,0,None,[],"December 18, 2015, 10:30am",2015-12-18 10:30:00,"December 28, 2015, 3:26pm","December 28, 2015, 3:26pm","December 28, 2015, 3:26pm"
121,15-1931,CLOSED,"Vehicle Stop Data for period September 1, 2015...",2015-12-28,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,None,title \ 0 R...,0,None,[],"December 28, 2015, 10:56am",2015-12-28 10:56:00,"December 31, 2015, 1:40pm","December 31, 2015, 1:40pm","December 31, 2015, 1:40pm"


In [26]:
"""def get_closing_time(msgs):
    #messages = row["msgs"]
    times = re.findall("[A-Z].{,10}[0-9]{,3}, [0-9]{4}, [0-9]{,2}:[0-9]*[pa]m", msgs)
    doc_updates = re.findall("Department|Document... Released|Request [A-Z].*",msgs )
    i=0
    for string in doc_updates:
        if "Closed" in string:
            break
        else:
            i+=1
    return times[i]

sd["Closing Time"] = sd["msgs"].apply(get_closing_time)
sd.head()"""

In [81]:
sd_police['Closing Time (dt)'] = pd.to_datetime(sd_police['Closing Time (str)'].astype(str))
sd_police.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,...,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time,close,closing,Closing Time (dt)
7,15-1817,CLOSED,File materials related to: P12010041171 P12050...,2015-12-07,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",02c. R...",San Diego,CA,...,title ...,0,None,[],"December 8, 2015, 11:40am",2015-12-08 11:40:00,"December 21, 2015, 10:48am","December 21, 2015, 10:48am","December 21, 2015, 10:48am",2015-12-21 10:48:00
49,15-1859,CLOSED,All records related to the demographic study c...,2015-12-11,Police,,Humberto Hernandez,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,title \ 0 Request ...,0,6254(c),[],"December 14, 2015, 6:07pm",2015-12-14 18:07:00,"January 12, 2016, 10:06am","January 12, 2016, 10:06am","January 12, 2016, 10:06am",2016-01-12 10:06:00
65,15-1875,CLOSED,Reports related to incident of 11/11/2015 wher...,2015-12-15,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",""02. R...",San Diego,CA,...,title \ 0 R...,0,None,"[Public"",""02. Released\nAll responsive documen...","December 16, 2015, 11:32am",2015-12-16 11:32:00,"January 13, 2016, 7:40am","January 13, 2016, 7:40am","January 13, 2016, 7:40am",2016-01-13 07:40:00
81,15-1891,CLOSED,Communications by the City of San Diego regard...,2015-12-17,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title ...,0,None,[],"December 18, 2015, 10:30am",2015-12-18 10:30:00,"December 28, 2015, 3:26pm","December 28, 2015, 3:26pm","December 28, 2015, 3:26pm",2015-12-28 15:26:00
121,15-1931,CLOSED,"Vehicle Stop Data for period September 1, 2015...",2015-12-28,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,0,None,[],"December 28, 2015, 10:56am",2015-12-28 10:56:00,"December 31, 2015, 1:40pm","December 31, 2015, 1:40pm","December 31, 2015, 1:40pm",2015-12-31 13:40:00


### Compute average time it takes for SD police department to complete a request

In [82]:
sd_police['days_until_completed'] = sd_police['Closing Time (dt)'] - sd_police['Submission Time (dt)']
sd_police.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,...,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time,close,closing,Closing Time (dt),days_until_completed
7,15-1817,CLOSED,File materials related to: P12010041171 P12050...,2015-12-07,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",02c. R...",San Diego,CA,...,0,None,[],"December 8, 2015, 11:40am",2015-12-08 11:40:00,"December 21, 2015, 10:48am","December 21, 2015, 10:48am","December 21, 2015, 10:48am",2015-12-21 10:48:00,12 days 23:08:00
49,15-1859,CLOSED,All records related to the demographic study c...,2015-12-11,Police,,Humberto Hernandez,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,0,6254(c),[],"December 14, 2015, 6:07pm",2015-12-14 18:07:00,"January 12, 2016, 10:06am","January 12, 2016, 10:06am","January 12, 2016, 10:06am",2016-01-12 10:06:00,28 days 15:59:00
65,15-1875,CLOSED,Reports related to incident of 11/11/2015 wher...,2015-12-15,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",""02. R...",San Diego,CA,...,0,None,"[Public"",""02. Released\nAll responsive documen...","December 16, 2015, 11:32am",2015-12-16 11:32:00,"January 13, 2016, 7:40am","January 13, 2016, 7:40am","January 13, 2016, 7:40am",2016-01-13 07:40:00,27 days 20:08:00
81,15-1891,CLOSED,Communications by the City of San Diego regard...,2015-12-17,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,0,None,[],"December 18, 2015, 10:30am",2015-12-18 10:30:00,"December 28, 2015, 3:26pm","December 28, 2015, 3:26pm","December 28, 2015, 3:26pm",2015-12-28 15:26:00,10 days 04:56:00
121,15-1931,CLOSED,"Vehicle Stop Data for period September 1, 2015...",2015-12-28,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,0,None,[],"December 28, 2015, 10:56am",2015-12-28 10:56:00,"December 31, 2015, 1:40pm","December 31, 2015, 1:40pm","December 31, 2015, 1:40pm",2015-12-31 13:40:00,3 days 02:44:00


In [86]:
print("Mean time for SD police dept to complete a request: ", sd_police['days_until_completed'].mean().round('1min'))

Mean time for SD police dept to complete a request:  12 days 16:08:00
